In [ ]:
Transfer Learning Homework 

Below you see a tutorial from Keras on using transfer learning. They train their models on half  the MNIST dataset digits (i.e. digits 0-4) and “transfer” the model to the second half (i.e, digits 5-9). 

Your homework is to train on all digits and make your own handwritten data set of the 5 characters {A, B, C, D, E}  and “transfer” your MNIST trained model over to the dataset you created. In other words, train on the full MNIST datasets (i.e. digits 0-9) and transfer on the {A, B, C, D, E} image dataset. 

Please do not use any other data resources from the web such as the emnist dataset. Figuring out the challenges of making your own handwritten character dataset is part of the exercise for this homework!   

Include the images you generated as part of your submission. Include any code used for the character image preprocessing. In a README file include details on your data generation process.

Code source: https://github.com/awslabs/keras-apache-mxnet/blob/master/examples/mnist_transfer_cnn.py


In [2]:
from __future__ import print_function
import datetime
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
import os
from keras.utils import img_to_array, load_img
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.model_selection import train_test_split
import numpy as np

now = datetime.datetime.now
batch_size = 128
epochs = 10
num_classes = 10
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

(x_train, y_train), (x_test, y_test) = mnist.load_data()

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

# create complete model
model = Sequential(feature_layers + classification_layers)

# train model for 5-digit classification [0..4]
train_model(model,
            (x_train, y_train),
            (x_test, y_test), num_classes=10)

# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False



folder = './Allen_Hoskins_HW4/letters HW4/'
x = np.array([])
y = np.array([])
for f in os.listdir(folder):
     if f.endswith('.png'):
        img = load_img(os.path.join(folder, f), color_mode='grayscale', target_size=(28, 28))
        img = img_to_array(img)
        label = ord(f[0])-ord("A")
        y = np.append(y,label)
        if len(x) == 0:
            x = np.array([img])
        else:
            x = np.vstack((x, [img]))

x = x.reshape(-1, 28, 28, 1)

x_train, x_test, y_train, y_test =train_test_split(x,y,test_size=0.25)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


model.pop()

classification_layers2 = [    Dense(128),    Activation('relu'),    Dropout(0.5),    Dense(5),    Activation('softmax')]

model2 = Sequential(feature_layers + classification_layers2)


# Freeze feature layers
for l in feature_layers:
    l.trainable = False

# Train model2 on letter dataset
train_model(model2, (x_train, y_train), (x_test, y_test), num_classes=5)



x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/10
  1/469 [..............................] - ETA: 2:41 - loss: 2.3082 - accuracy: 0.1016

2023-03-05 18:43:58.405909: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - ETA: 0s - loss: 2.2864 - accuracy: 0.1484

2023-03-05 18:44:06.056764: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 9s 18ms/step - loss: 2.2864 - accuracy: 0.1484 - val_loss: 2.2669 - val_accuracy: 0.2190
Epoch 2/10
469/469 [==============================] - 8s 18ms/step - loss: 2.2567 - accuracy: 0.1960 - val_loss: 2.2321 - val_accuracy: 0.2921
Epoch 3/10
183/469 [==========>...................] - ETA: 4s - loss: 2.2336 - accuracy: 0.2317

KeyboardInterrupt: 